<a href="https://colab.research.google.com/github/alscop/ESAA-25-2/blob/main/YB_1107_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%ED%9A%8C%EA%B7%801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 단순 선형 회귀 분석**
- 전복의 나이를 예측하는 선형회귀모델을 생성하세요.
- 전복의 ‘성별’, ‘키’, ‘지름’, ‘높이’, ‘전체무게’, ‘몸통무게’, ‘내장무게’, ‘껍질무게’를 이용해 ‘껍질의 고리 수’를 예측한 뒤, **예측된 ‘껍질의 고리 수’에 1.5를 더하면 전복의 나이**가 됩니다.

In [3]:
# 기본 모듈 불러오기
import numpy as np
import pandas as pd

**1) 데이터 load 및 변형**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/EWHA/ESAA/dataset/abalone.csv')
data.head()
print(data.shape)

# 성별 M은 Male, F는 Female, I는 Infant 이므로 따로 열 만들기
for label in "MFI":
    data[label] = data["Sex"] == label
data.drop('Sex', axis=1, inplace=True)

(4177, 9)


**2) X, y 선택**
: y는 Rings열, X는 Rings열을 제외한 나머지를 선택하되 전부 실수가 되도록 한다.

In [29]:
# X,y 데이터 선택
y = data['Rings']
X = data.drop('Rings', axis=1)
X = X.astype(float)

print(X.dtypes)

Length            float64
Diameter          float64
Height            float64
Whole weight      float64
Shucked weight    float64
Viscera weight    float64
Shell weight      float64
M                 float64
F                 float64
I                 float64
dtype: object


전부 실수로 변경된 것 확인함

 **3) train/test set 분리**

In [10]:
# 필요한 모듈 불러오기
from sklearn.model_selection import train_test_split

In [30]:
# train과 test set 분리 (train:test = 7:3 비율로)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=156
    )

**4) 선형회귀모델 생성, 모델 예측치 구하기**

In [8]:
#필요한 모듈 불러오기
from sklearn.linear_model import LinearRegression

In [31]:
#선형회귀모델 생성 및 훈련
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [32]:
# 모델 예측치 구하기
y_preds = lr.predict(X_test)

# 모델 예측치를 활용해 최종적으로 전복의 나이를 예측
predicted_ages = y_preds + 1.5
predicted_ages

array([12.93605621, 10.31680051, 10.05943354, ..., 12.48281237,
       10.79055844,  7.88127031])

**5) 모델 평가: MSE, RMSE, R2 score, corr 구하기**

In [33]:
#필요한 모듈 불러오기
from sklearn.metrics import mean_squared_error, r2_score

- MSE, RMSE

In [34]:
#mse, rmse
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('mse값:',mse, '\nrmse 값:', rmse)

mse값: 5.192570674774858 
rmse 값: 2.278721280625355


- R2 score

In [28]:
#R2 score 측정
R2score = r2_score(y_test, y_preds)
print('R2score값:',R2score)

R2score값: 0.4863291321197166


- 회귀 절편값

In [24]:
#회귀 절편 값
print('절편 값:', lr.intercept_)

절편 값: 3.248395545002505


- 회귀 계수 값

In [35]:
#회귀 계수 값
print('회귀 계수값:', np.round(lr.coef_, 2))

회귀 계수값: [ -2.67  11.16  24.25   9.58 -20.23 -11.63   6.59   0.27   0.24  -0.51]


- 상관계수

Hint: corr 함수 이용.

In [37]:
# 상관계수 구하기
corr = y_test.corr(pd.Series(y_preds))
print('상관계수값:', corr)

상관계수값: 0.036698234798395186


# **2. Polynomial features**

In [5]:
# PolynomialFeatures 라이브러리 호출
from sklearn.preprocessing import PolynomialFeatures

In [38]:
# 임의 데이터 생성

X = np.arange(6).reshape(3, 2)

df =  pd.DataFrame(X)
df.columns = ['x_1','x_2']
df

,x_1,x_2
0,0,1
1,2,3
2,4,5


In [39]:
# 차원은 2로 설정
poly = PolynomialFeatures(degree=2)

# fit_transform 메소드를 통해 데이터 변환
X_poly = poly.fit_transform(df)

# PolynomialFeatures로 변환 된 데이터를 데이터 프레임 형태로 변환
df_poly = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(df.columns))

df_poly

,1,x_1,x_2,x_1^2,x_1 x_2,x_2^2
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0


In [41]:
# df_poly의 컬럼을 1,x1,x2,x1^2,x1*x2,x2^2 로 변경
df_poly.columns = ['1', 'x1', 'x2','x1^2', 'x1*x2', 'x2^2']
df_poly

,1,x1,x2,x1^2,x1*x2,x2^2
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0


칼럼명 변경된 것 확인 가능.